## 🤗Transformers - Generating Articles from Paper's Abstracts using T5 Model
This notebook uses T5 model - A Sequence to Sequence model fully capable to perform any text to text tasks. What does it mean - It means that T5 model can take any input text and convert it into any output text. Such Text to Text conversion is useful in NLP tasks like language translation, summarization etc.

In this notebook, we will take paper's abstracts as our input text and paper's title as output text and feed it to T5 model. So,let's dive in...



We will install dependencies and work with latest stable pytorch 1.6

!pip uninstall torch torchvision -y

!pip install torch==1.6.0+cu101 torchvision==0.7.0+cu101 -f https://download.pytorch.org/whl/torch_stable.html

!pip install -U transformers

!pip install -U simpletransformers  

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
import os, psutil  

def cpu_stats():
    pid = os.getpid()
    py = psutil.Process(pid)
    memory_use = py.memory_info()[0] / 2. ** 30
    return 'memory GB:' + str(np.round(memory_use, 2))

In [3]:
cpu_stats()

'memory GB:0.08'

In [4]:
import json

data_file = '../../arxiv_download/tools/arxiv-metadata-oai-snapshot.json'

def get_metadata():
    with open(data_file, 'r') as f:
        for line in f:
            yield line

In [5]:
metadata = get_metadata()
for paper in metadata:
    paper_dict = json.loads(paper)
    print('Title: {}\n\nAbstract: {}\nRef: {}'.format(paper_dict.get('title'), paper_dict.get('abstract'), paper_dict.get('journal-ref')))
#     print(paper)
    break

Title: Calculation of prompt diphoton production cross sections at Tevatron and
  LHC energies

Abstract:   A fully differential calculation in perturbative quantum chromodynamics is
presented for the production of massive photon pairs at hadron colliders. All
next-to-leading order perturbative contributions from quark-antiquark,
gluon-(anti)quark, and gluon-gluon subprocesses are included, as well as
all-orders resummation of initial-state gluon radiation valid at
next-to-next-to-leading logarithmic accuracy. The region of phase space is
specified in which the calculation is most reliable. Good agreement is
demonstrated with data from the Fermilab Tevatron, and predictions are made for
more detailed tests with CDF and DO data. Predictions are shown for
distributions of diphoton pairs produced at the energy of the Large Hadron
Collider (LHC). Distributions of the diphoton pairs from the decay of a Higgs
boson are contrasted with those produced from QCD processes at the LHC, showing
tha

**We will take last 5 years ArXiv papers (2016-2021) due to Kaggle'c compute limits**

In [6]:
titles = []
abstracts = []
years = []
metadata = get_metadata()
for paper in metadata:
    paper_dict = json.loads(paper)
    ref = paper_dict.get('journal-ref')
    try:
        year = int(ref[-4:]) 
        if 2010 < year < 2021:
            years.append(year)
            titles.append(paper_dict.get('title'))
            abstracts.append(paper_dict.get('abstract'))
    except:
        pass 

len(titles), len(abstracts), len(years)

(42014, 42014, 42014)

In [7]:
papers = pd.DataFrame({
    'title': titles,
    'abstract': abstracts,
    'year': years
})
papers.head()

,title,abstract,year
0,The World as Evolving Information,This paper discusses the benefits of describ...,2012
1,A unified analysis of the reactor neutrino pro...,We present in this article a detailed quanti...,2013
2,Heat Equations and the Weighted $\bar\partial$...,The purpose of this article is to establish ...,2012
3,The KATRIN sensitivity to the neutrino mass an...,The aim of the KArlsruhe TRItium Neutrino ex...,2011
4,"Penguin-mediated B_(d,s)->VV decays and the Bs...","In this letter, we propose three different s...",2011


In [8]:
del titles, abstracts, years

In [9]:
cpu_stats()

'memory GB:0.14'

 **We will use `simpletransformers` library to train a T5 model**

In [10]:
import logging
logging.basicConfig(level=logging.INFO)
transformers_logger = logging.getLogger("transformers")
transformers_logger.setLevel(logging.WARNING)


**Simpletransformers implementation of T5 model expects a data to be a dataframe with 3 columns:**
`<prefix>, <input_text>, <target_text>`
* `<prefix>`: A string indicating the task to perform. (E.g. "question", "stsb")
* `<input_text>`: The input text sequence (we will use Paper's abstract as `input_text`  )
* `<target_text`: The target sequence (we will use Paper's title as `output_text` )
    
    
 You can read about the data format:  https://github.com/ThilinaRajapakse/simpletransformers#t5-transformer

In [11]:
papers = papers[['title','abstract']]
papers.columns = ['target_text', 'input_text']
papers = papers.dropna()

In [12]:
eval_df = papers.sample(frac=0.2, random_state=101)
train_df = papers.drop(eval_df.index)

In [13]:
train_df.shape, eval_df.shape

((33611, 2), (8403, 2))

**We will training out T5 model with very bare minimum `num_train_epochs=4`, `train_batch_size=16` to  fit into Kaggle's compute limits**

In [14]:
import logging

import pandas as pd
from simpletransformers.t5 import T5Model

logging.basicConfig(level=logging.INFO)
transformers_logger = logging.getLogger("transformers")
transformers_logger.setLevel(logging.WARNING)

train_df['prefix'] = "summarize"
eval_df['prefix'] = "summarize"


model_args = {
    "reprocess_input_data": True,
    "overwrite_output_dir": True,
    "max_seq_length": 512,
    "train_batch_size": 16,
    "num_train_epochs": 4,
}

# Create T5 Model
model = T5Model("t5", "t5-small", args=model_args, use_cuda=True)

# Train T5 Model on new task
model.train_model(train_df)

# Evaluate T5 Model on new task
results = model.eval_model(eval_df)

# Predict with trained T5 model
#print(model.predict(["convert: four"]))

INFO:simpletransformers.t5.t5_utils: Creating features from dataset file at cache_dir/


INFO:simpletransformers.t5.t5_utils: Saving features into cached file cache_dir/t5-small_cached_51233611

INFO:simpletransformers.t5.t5_model: Training started
Using Adafactor for T5




INFO:simpletransformers.t5.t5_model: Training of t5-small model complete. Saved to outputs/.
INFO:simpletransformers.t5.t5_utils: Creating features from dataset file at cache_dir/


INFO:simpletransformers.t5.t5_utils: Saving features into cached file cache_dir/t5-small_cached_5128403



INFO:simpletransformers.t5.t5_model:{'eval_loss': 2.0226785508254728}



In [20]:
results

{'eval_loss': 2.0226785508254728}

## And We're Done ! 
**Let's see how our model performs in generating paper's titles**

In [16]:
random_num = 350
actual_title = eval_df.iloc[random_num]['target_text']
actual_abstract = ["summarize: "+eval_df.iloc[random_num]['input_text']]
predicted_title = model.predict(actual_abstract)

print(f'Actual Title: {actual_title}')
print(f'Predicted Title: {predicted_title}')
print(f'Actual Abstract: {actual_abstract}')



Actual Title: Production of the Exotic $1^{--}$ Hadrons $\phi(2170)$, X(4260) and
  $Y_b(10890)$ at the LHC and Tevatron via the Drell-Yan Mechanism
Predicted Title: ['Drell-Yan production cross sections and differential distributions in the transverse momentum and rapid']
Actual Abstract: ['summarize:   We calculate the Drell-Yan production cross sections and differential\ndistributions in the transverse momentum and rapidity of the $J^{PC}=1^{--}$\nexotic hadrons $\\phi(2170)$, X(4260) and $Y_b(10890)$ at the hadron colliders\nLHC and the Tevatron. These hadrons are tetraquark (four-quark) candidates,\nwith a hidden $s\\bar{s}$, $c\\bar{c}$ and $b\\bar{b}$ quark pair, respectively.\nIn deriving the distributions and cross sections, we include the order\n$\\alpha_s$ QCD corrections, resum the large logarithms in the small transverse\nmomentum region in the impact-parameter formalism, and use the state of the art\nparton distribution functions. Taking into account the data on the prod

In [17]:
random_num = 478
actual_title = eval_df.iloc[random_num]['target_text']
actual_abstract = ["summarize: "+eval_df.iloc[random_num]['input_text']]
predicted_title = model.predict(actual_abstract)

print(f'Actual Title: {actual_title}')
print(f'Predicted Title: {predicted_title}')
print(f'Actual Abstract: {actual_abstract}')


Actual Title: Spectral analysis of photonic crystals made of thin rods
Predicted Title: ['Transverse electric modes and transverse electric modes in photonic crystals']
Actual Abstract: ['summarize:   In this paper we address the question how to design photonic crystals that\nhave photonic band gaps around a finite number of given frequencies. In such\nmaterials electromagnetic waves with these frequencies can not propagate; this\nmakes them interesting for a large number of applications. We focus on crystals\nmade of periodically ordered thin rods with high contrast dielectric\nproperties. We show that the material parameters can be chosen in such a way\nthat transverse magnetic modes with given frequencies can not propagate in the\ncrystal. At the same time, for any frequency belonging to a predefined range\nthere exists a transverse electric mode that can propagate in the medium. These\nresults are related to the spectral properties of a weighted Laplacian and of\nan elliptic opera

In [18]:
random_num = 999
actual_title = eval_df.iloc[random_num]['target_text']
actual_abstract = ["summarize: "+eval_df.iloc[random_num]['input_text']]
predicted_title = model.predict(actual_abstract)

print(f'Actual Title: {actual_title}')
print(f'Predicted Title: {predicted_title}')
print(f'Actual Abstract: {actual_abstract}')


Actual Title: Dynamic Parameter Allocation in Parameter Servers
Predicted Title: ['Using Dynamic Parameter Allocation to Improve Parameter Access Locality']
Actual Abstract: ['summarize:   To keep up with increasing dataset sizes and model complexity, distributed\ntraining has become a necessity for large machine learning tasks. Parameter\nservers ease the implementation of distributed parameter management---a key\nconcern in distributed training---, but can induce severe communication\noverhead. To reduce communication overhead, distributed machine learning\nalgorithms use techniques to increase parameter access locality (PAL),\nachieving up to linear speed-ups. We found that existing parameter servers\nprovide only limited support for PAL techniques, however, and therefore prevent\nefficient training. In this paper, we explore whether and to what extent PAL\ntechniques can be supported, and whether such support is beneficial. We propose\nto integrate dynamic parameter allocation int

In [19]:
import random

document = []

for _ in range(250):
    data = dict()

    random_idx = random.randint(0, len(eval_df)-1)
    
    actual_abstract = eval_df.iloc[random_idx]['input_text']
    actual_title = eval_df.iloc[random_idx]['target_text']

    # Predict with trained T5 model
    predicted_title = model.predict(["summarize: " + actual_abstract])[0]

    print(f'Actual Title: {actual_title}')
    print(f'Predicted Title: {predicted_title}')
    print(f'Actual Abstract: {actual_abstract}')

    data['true_title'] = actual_title
    data['pred_title'] = predicted_title
    data['abstract'] = actual_abstract

    document.append(data)

import json
with open('result.json', 'w') as f:
    json.dump(document, f)



Actual Title: Performance Requirements of Advanced Healthcare Services over Future
  Cellular Systems
Predicted Title: The Technical Requirements of Future Cellular Networks
Actual Abstract:   The fifth generation (5G) of communication systems has ambitious targets of
data rate, end-to-end latency, and connection availability, while the
deployment of a new flexible network architecture will spur new applications.
E-health and mobile health (m-health) solutions will meet the increasing demand
of new, sustainable, and more accessible services beneficial to both
practitioners and the rapidly aging population. This paper aims at defining the
technical requirements of future cellular networks to support a variety of
advanced healthcare services (e.g., smart hospital, telesurgery, connected
ambulances, and monitoring). While 5G will be able to satisfy these
requirements, it will also pave the way for future e- and m-health in the
sixth-generation (6G) cellular networks.




Actual Title: Searching for CPT Violation with Neutral-Meson Oscillations
Predicted Title: CPT violation in neutral-meson oscillations
Actual Abstract:   A general technique is presented for treating CPT violation in neutral-meson
oscillations. The effective field theory for a complex scalar with
CPT-violating operators of arbitrary mass dimension is incorporated in the
formalism for the propagation and mixing of neutral mesons. Observable effects
are discussed, and first measurements of CPT-violating operators of dimension
five are extracted from existing experimental results.




Actual Title: On G-Fano threefolds
Predicted Title: On Fano threefolds with a "minimal" action of a finite
Actual Abstract:   We study Fano threefolds with~terminal singularities admitting a "minimal"
action of a finite group. We prove that under certain additional assumptions
such a variety does not contain planes. We also obtain an upper bounds of the
number of singular points of certain Fano threefolds with~terminal factorial
singularities.




Actual Title: Modeling Data Containing Outliers using ARIMA Additive Outlier
  (ARIMA-AO)
Predicted Title: Detection and Correction of Additive Outlier on the Model ARIMA
Actual Abstract:   The aim this study is discussed on the detection and correction of data
containing the additive outlier (AO) on the model ARIMA (p, d, q). The process
of detection and correction of data using an iterative procedure popularized by
Box, Jenkins, and Reinsel (1994). By using this method we obtained an ARIMA
models were fit to the data containing AO, this model is added to the original
model of ARIMA coefficients obtained from the iteration process using
regression methods. This shows that there is an improvement of forecasting
error rate data.




Actual Title: Planarity and Street Network Representation in Urban Form Analysis
Predicted Title: Measurement of Nonplanarity of Urban Street Networks
Actual Abstract:   Models of street networks underlie research in urban travel behavior,
accessibility, design patterns, and morphology. These models are commonly
defined as planar, meaning they can be represented in two dimensions without
any underpasses or overpasses. However, real-world urban street networks exist
in three-dimensional space and frequently feature grade separation such as
bridges and tunnels: planar simplifications can be useful but they also impact
the results of real-world street network analysis. This study measures the
nonplanarity of drivable and walkable street networks in the centers of 50
cities worldwide, then examines the variation of nonplanarity across a single
city. It develops two new indicators - the Spatial Planarity Ratio and the Edge
Length Ratio - to measure planarity and describe infrastructure and

...


Actual Title: Generalized Knudsen Number for Unsteady Fluid Flow
Predicted Title: Scaling behavior of an unsteady flow
Actual Abstract:   We explore the scaling behavior of an unsteady flow that is generated by an
oscillating body of finite size in a gas. If the gas is gradually rarefied, the
Navier-Stokes equations begin to fail and a kinetic description of the flow
becomes more appropriate. The failure of the Navier-Stokes equations can be
thought to take place via two different physical mechanisms: either the
continuum hypothesis breaks down as a result of a finite size effect; or local
equilibrium is violated due to the high rate of strain. By independently tuning
the relevant linear dimension and the frequency of the oscillating body, we can
experimentally observe these two different physical mechanisms. All the
experimental data, however, can be collapsed using a single dimensionless
scaling parameter that combines the relevant linear dimension and the frequency
of the body. Thi


Actual Title: Black hole masses of weak emission line quasars based on continuum fit
  method
Predicted Title: Optical-ultraviolet spectral energy distribution of weak emission-line
Actual Abstract:   We studied optical-ultraviolet spectral energy distribution of 10 weak
emission-line quasars (WLQs) which lie at redshifts z = 0.19 and 1.43 < z <
3.48. The theoretical models of their accretion disk continua are created based
on the Novikov-Thorne equations. It allows us to estimate masses of their
supermassive black holes (MBH) and accretion rates. We determined the virial
factor for WLQs and note its anti-correlation with the full width at half
maximum (FWHM) of Hbeta emission-line (f propto FWHM^(alpha), alpha = (-1.34
+/- 0.37)). By comparison with the previously estimated BH masses, the
underestimation of MBH is noticed with a mean factor 4-5 which depends on the
measured full width. We proposed the new formula to estimate MBH in WLQs based
on their observed FWHM (H beta) and lumin


Actual Title: Theoretical Investigation: 2D N-Graphdiyne Nanosheets as Promising Anode
  Materials for Li/Na Rechargeable Storage Devices
Predicted Title: Newly fabricated N-graphdiyne monolayers as anode
Actual Abstract:   N-graphdiyne monolayers, a set of carbon-nitride nanosheets, have been
synthesized recently through the polymerization of triazine- and pyrazine-based
monomers. Since the two-dimensional nano-structures are mainly composed of
light-weight nonmetallic elements including carbon and nitrogen, they might be
able to provide high storage capacities for rechargeable cells. In this study,
we used extensive first principle calculations such as electronic density of
states, band structure, adsorption energy, open-circuit voltage, nudged-elastic
band and charge analyses to investigate the application of the newly fabricated
N-graphdiyne monolayers as the anode material for Li/Na/Mg ion batteries. Our
calculations suggest that while Mg foreign atoms poorly interact with
monola


Actual Title: Five Proofs of Chernoff's Bound with Applications
Predicted Title: On the Bound of Chernoff
Actual Abstract:   We discuss five ways of proving Chernoff's bound and show how they lead to
different extensions of the basic bound.




Actual Title: Charge Transfer Kinetics at the Solid-Solid Interface in Porous
  Electrodes
Predicted Title: Interfacial charge transfer in carbon-coated LiFePO4 porous electrode
Actual Abstract:   Interfacial charge transfer is widely assumed to obey Butler-Volmer kinetics.
For certain liquid-solid interfaces, Marcus-Hush-Chidsey theory is more
accurate and predictive, but it has not been applied to porous electrodes. Here
we report a simple method to extract the charge transfer rates in carbon-coated
LiFePO4 porous electrodes from chronoamperometry experiments, obtaining curved
Tafel plots that contradict the Butler-Volmer equation but fit the
Marcus-Hush-Chidsey prediction over a range of temperatures. The fitted
reorganization energy matches the Born solvation energy for electron transfer
from carbon to the iron redox site. The kinetics are thus limited by electron
transfer at the solid-solid (carbon-LixFePO4) interface, rather than by ion
transfer at the liquid-solid interface, as